### Gold and Silver

In [31]:
import warnings
import pandas as pd
import numpy as np
import json
import yfinance as yf
import statsmodels.api as sm
import pandas_gbq
import sys, os
from google.cloud import bigquery
from google.oauth2 import service_account
import datetime
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import row
from bokeh.io import output_file, output_notebook
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Slope
from bokeh.models import PolyAnnotation
from bokeh.models.widgets import Tabs, Panel
import requests
from google.cloud import bigquery
from google.oauth2 import service_account
from multiprocessing.pool import ThreadPool
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # default='warn'

In [32]:
cheked_year = '2007'
cheked_year_end = '2021'
warnings.filterwarnings('ignore')
# *****************************SQL params ***************
credentials = service_account.Credentials.from_service_account_file('gadgets_database_key.json')
project_id = 'buoyant-apogee-281013'
client = bigquery.Client(credentials=credentials, project=project_id)
table_id_guru = "buoyant-apogee-281013.FIN.GURU"
table_id_yahoo_adj_close = "buoyant-apogee-281013.FIN.Summary"
job_config = bigquery.QueryJobConfig()

exchange = ('M2 Money Stock', 'Gold Fixing Price 10:30 GMT in London Bullion Market, based in USD', \
           '10-year Treasury Inflation-Indexed Security, Constant Maturity')

sql_guru_req = f"""
SELECT *
FROM `buoyant-apogee-281013.FIN.Economic_Indicators`
WHERE Indicator in {exchange}
"""

sql_data_guru = pandas_gbq.read_gbq(sql_guru_req, project_id=project_id)
sql_data_guru = sql_data_guru.set_index('Indicator')
sql_data_guru['description'] # обращение к описанию отрасли
final_df = pd.DataFrame()

for indicator in sql_data_guru.index.tolist():
    io = json.loads(sql_data_guru.loc[indicator].loc['data'].replace('"', '~').replace("None", '"0"').replace("'", '"').replace("True", '"True"').replace("False", '"False"'))
    pr_df = pd.DataFrame(io, columns=['year', indicator]).set_index('year')
    final_df = final_df.join(pr_df, how='outer')
final_df.index = pd.to_datetime(final_df.index)
final_df = final_df[(final_df.index >= cheked_year) & (final_df.index <= cheked_year_end)]

gold = pd.DataFrame()
gold = yf.download('GC=F', start='2007-01-01')['Adj Close']
#gold = gold.asfreq(freq='M', method='ffill')
silver = pd.DataFrame()
silver = yf.download('SI=F', start='2007-01-01')['Adj Close']
#silver = silver.asfreq(freq='M', method='ffill')
btc = pd.DataFrame()
btc = yf.download('BTC-USD', start='2018-01-01')['Adj Close']
btc = btc.asfreq(freq='B')
sp500 = pd.DataFrame()
sp500 = yf.download('^GSPC', start='2007-01-01')['Adj Close']
gdaxi = pd.DataFrame()
gdaxi = yf.download('EWG', start='2007-01-01')['Adj Close']
sti = pd.DataFrame()
sti = yf.download('EWS', start='2007-01-01')['Adj Close']
imoex = pd.DataFrame()
imoex = yf.download('ERUS', start='2007-01-01')['Adj Close']
#final_df.tail()

Downloading: 100%|██████████| 3/3 [00:00<00:00,  3.99rows/s]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Графики

In [33]:
output_notebook()
m2 = final_df['M2 Money Stock'].dropna()
x = figure(title='Цепной индекс M2 money stock + Gold Fixing Price', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
x.line(m2.index, m2/m2[0], color='green', line_width=1, legend='M2 money stock')
x.line(gold.index, gold/gold[0], color='orange', line_width=1, legend='Gold Fixing Price')
x.line(silver.index, silver/silver[0], color='cornflowerblue', line_width=1, legend='Silver Price')
x.line(btc.index, btc/btc[0], color='salmon', line_width=1, legend='BTC Price')
x.legend.location = 'top_left'
treasure = final_df['10-year Treasury Inflation-Indexed Security, Constant Maturity'].dropna()
y = figure(title='Цепной индекс Gold Fixing Price + 10-year Treasury', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
y.line(treasure.index, treasure/treasure[0], color='green', line_width=1, legend='10-year Treasury')
y.line(gold.index, gold/gold[0], color='orange', line_width=1, legend='Gold Fixing Price')
y.line(silver.index, silver/silver[0], color='cornflowerblue', line_width=1, legend='Silver Price')
y.line(btc.index, btc/btc[0], color='salmon', line_width=1, legend='BTC Price')
y.legend.location = 'top_left'
show(row(x,y))
c = figure(title='Цепной индекс Gold Price + Silver Price', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
c.line(gold.index, gold/gold[0], color='orange', line_width=1, legend='Gold Price')
c.line(silver.index, silver/silver[0], color='cornflowerblue', line_width=1, legend='Silver Price')
c.line(btc.index, btc/btc[0], color='salmon', line_width=1, legend='BTC Price')
c.legend.location = 'top_left'
d = figure(title='Gold Price + Silver Price', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
d.line(gold.index, gold, color='orange', line_width=1, legend='Gold Price')
d.legend.location = 'top_left'
y_column2_range = 'Silver Price'+ "_range"
d.extra_y_ranges = {y_column2_range: Range1d(start=(gold/silver).min() * (1 - 0.8), end=(gold/silver).max() * (1 + 0.1))}
d.add_layout(LinearAxis(y_range_name=y_column2_range), "right")
d.line(silver.index, silver, legend='Silver Price', line_width=1, y_range_name=y_column2_range,  color="cornflowerblue")
d.line(gold.index, gold/silver, color='grey', line_width=0.5, y_range_name=y_column2_range, legend='Gold/silver Price')
show(row(c,d))
print("gold/silver")
print((gold/silver).describe())
print('***************************')
e1 = figure(title='S&P500 in grams of Gold', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
e1.line(gold.index, (sp500/gold)*31.1035, color='green', line_width=1, legend='S&P500 in grams of Gold')
e1.legend.location = 'top_left'
e2 = figure(title='GDAXI in grams of Gold', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
e2.line(gold.index, (gdaxi/gold)*31.1035, color='blue', line_width=1, legend='EWG in grams of Gold')
e2.legend.location = 'top_left'
e3 = figure(title='Singapore in grams of Gold', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
e3.line(gold.index, (sti/gold)*31.1035, color='red', line_width=1, legend='EWS in grams of Gold')
e3.legend.location = 'top_left'
e4 = figure(title='IMOEX in grams of Gold', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
e4.line(gold.index, (imoex/gold)*31.1035, color='black', line_width=1, legend='ERUS in grams of Gold')
e4.legend.location = 'top_left'
e1_panel = Panel(child=e1, title='S&P500 in Gold')
e2_panel = Panel(child=e2, title='GDAXI in Gold')
e3_panel = Panel(child=e3, title='Singapore in Gold')
e4_panel = Panel(child=e4, title='IMOEX in Gold')
tabs1 = Tabs(tabs=[e1_panel, e2_panel, e3_panel, e4_panel])
f1 = figure(title='S&P500 in grams of Silver', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
f1.line(silver.index, (sp500/silver)*31.1035, color='green', line_width=1, legend='S&P500 in grams of Silver')
f1.legend.location = 'top_left'
f2 = figure(title='GDAXI in grams of Silver', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
f2.line(silver.index, (gdaxi/silver)*31.1035, color='blue', line_width=1, legend='EWG in grams of Silver')
f2.legend.location = 'top_left'
f3 = figure(title='Singapore in grams of Silver', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
f3.line(silver.index, (sti/silver)*31.1035, color='red', line_width=1, legend='EWS in grams of Silver')
f3.legend.location = 'top_left'
f4 = figure(title='IMOEX in grams of Silver', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
f4.line(silver.index, (imoex/silver)*31.1035, color='black', line_width=1, legend='ERUS in grams of Silver')
f4.legend.location = 'top_left'
f1_panel = Panel(child=f1, title='S&P500 in Silver')
f2_panel = Panel(child=f2, title='GDAXI in Silver')
f3_panel = Panel(child=f3, title='Singapore in Silver')
f4_panel = Panel(child=f4, title='IMOEX in Silver')
tabs2 = Tabs(tabs=[f1_panel, f2_panel, f3_panel, f4_panel])
show(row(tabs1,tabs2))
g1 = figure(title='S&P500 in BTC', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
g1.line(btc.index, sp500['2018':]/btc, color='green', line_width=1, legend='S&P500 in BTC')
g1.legend.location = 'top_left'
g2 = figure(title='GDAXI in BTC', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
g2.line(btc.index, (gdaxi['2018':]/btc)*31.1035, color='blue', line_width=1, legend='EWG in BTC')
g2.legend.location = 'top_left'
g3 = figure(title='Singapore in BTC', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
g3.line(btc.index, (sti['2018':]/btc)*31.1035, color='red', line_width=1, legend='EWS in BTC')
g3.legend.location = 'top_left'
g4 = figure(title='IMOEX in BTC', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
g4.line(btc.index, (imoex['2018':]/btc)*31.1035, color='black', line_width=1, legend='ERUS BTC')
g4.legend.location = 'top_left'
g1_panel = Panel(child=g1, title='S&P500 in BTC')
g2_panel = Panel(child=g2, title='GDAXI in BTC')
g3_panel = Panel(child=g3, title='Singapore in BTC')
g4_panel = Panel(child=g4, title='IMOEX in BTC')
tabs3 = Tabs(tabs=[g1_panel, g2_panel, g3_panel, g4_panel])
show(tabs3)

Loading BokehJS ...

gold/silver
count    3760.000000
mean       67.738202
std        13.496947
min        31.995078
25%        56.328029
50%        68.396827
75%        76.288586
max       125.888376
Name: Adj Close, dtype: float64
***************************
